# Tercer sesión

## Agenda

 1. Excepciones
 2. Comprehensions (list, set, dict y generador)
 3. Generadores y corrutinas
 4. Lambdas
 5. Módulos
 6. Paquetes
 7. Distribuyendo el código


## 1. Excepciones

### Sobre el manejo de errores *[contexto]*

Todo programa que sea propiamente escrito debe de tener una estrategia definida de manejo y prevención de errores, verificar los argumentos a sus funciones, prevenir efectos secundarios indeseados entre otra muchas otras cosas.

A groso modo existen dos tipos de condiciones que pueden tener al estar pensando en como lidiamos con condiciones excepcionales, estas pueden ser **fatales** o **no-fatales** donde cada una de esos tipos de condiciones pueden ser *esperadas* o *inesperadas*.

Algunas veces realmente se espera que el programa termine lo más pronto posible, como por ejemplo cuando recibe argumentos erróneos para el programa, otras veces la condición que no se considera optima para la funcionalidad deseada se puede corregir ahí mismo y es una apuesta bastante segura que existen condiciones inesperadas, las cuales muy probablemente serán fatales y terminaran la ejecución del programa ó en el caso de que sean no-fatales muy probablemente tendrán efectos secundarios indeseados y casi siempre terminan como errores difíciles de corregir o dicho de otra forma *bugs* complicados de debuggear, dependiendo de la organización y estructura del código así como la posibilidad de replicar las condiciones excepcionales.

Cabe destacar que la buena selección de estructuras de datos y la organización del código representan la barrera número uno para evitarnos un montón de problemas a futuro, en el casó que se tenga en mente mantener el código que se escribe en el futuro cercanos.

### Estrategias para el manejo de errores *[contexto]*

Desde una perspectiva simple existen dos *"escuelas"* del manejo de errores estás son: **look before you leap (LBYL)** / *mira antes de saltar* y **easier to ask for forgiveness than permission (EAFP)** / *más fácil pedir perdón que permiso*, en general la comunidad de python tiende a la segunda *(EAFP)* ya que el lenguaje esta diseñado para facilitar esa forma, lo cual casi siempre requiere un buen soporte al manejo de excepciones, esto no quiere decir que nunca de debe de usar *LBYL*, siempre es buena idea en casos donde se requiere especial cautela con las condiciones excepcionales, evitar tener efectos secundarios indeseados o que simplemente aporte mayor claridad al código.


#### LBYL (look before you leap) *[contexto]*

La estrategia look before you leap se basa en prevenir condiciones excepcionales esperadas, como por ejemplo:

```python
# name proviene de algun lugar misterioso
if not isinstance(name, str):
   name = "S/N"

print("Mi nombre es " + name)
```

Un caso común es el de verifica argumentos del programa como en:


```python
import sys

num = sys.argv[1]

if num.isdigit():
   num = int(num)
else:
   print("El argumento uno solo puede tener numeros")
   sys.exit(-1) # termina el programa
   
```

En los dos casos primero validamos que los argumentos sean los esperados luego operamos con los mismos, en el ejemplo dos, verificamos que el argumento número uno sea un número, dado que siempre se reciben como cadena, primero verificamos que la cadena contenga solo caracteres numéricos luego procedemos a convertirlo a un entero, dado que esta dentro del bloque del if garantizando que no va a ocurrir problema alguno al intentar hacer la conversión.

#### EAFP (easier to ask for forgiveness than permission) *[contexto]*

La estrategia de EAFP se basa en aportar la mayor cantidad de código que refiere a la funcionalidad de forma continua y dejar los casos extraños en bloques aparte, en espera de la excepción en particular o un bloque que captura todos los tipos de errores, para que EAFP pueda existir es necesaria la construcción de excepciones, con bloques de try/except para hacer una analogía a los ejemplos anteriores esta sería otra forma de lidiar con los argumentos que se esperan:

```python
try:
    print("Mi nombre es " + name)
except TypeError: # perdon!! :{ 
    print("Mi nombre es S/N")
```

Siguiendo con la idea de los argumentos de un programa:

```python
try:
    num = int(sys.argv[1])
except ValueError: # perdon! Fue sin querer! :{ 
    print("El argumento uno solo puede tener numeros")
    sys.exit(-1) # termina el programa
```

### try/exept

La construcción básica para el manejo de excepciones es el bloque try/except donde tiene la siguiente estructura:

```python
try:
   # bloque de codigo con funcion primaria
except TIPO_DE_EXCEPTION as ERR:
    # bloque de codigo con funcion para recuperar la excepcion y
    # se puede hacer uso de la variable ERR
```

In [9]:
try:
    print("PyMTY " + {})
except TypeError as err:
    print("Se capturo el error: {}".format(err))
    print("No puedes concatenar cadenas y diccionarios!")

Se capturo el error: Can't convert 'dict' object to str implicitly
No puedes concatenar cadenas y diccionarios!


### raise

Hasta el momento hemos visto como capturar excepciones, pero no se ha mencionado como nosotros mismo podemos levantar nuestras propias excepciones, para eso se ocupa la palabra reservada `raise` seguido de un objeto que herede de `Exception` o alguna otra excepción más especifica que se relacione, como `TypeError` o `NotImplementedError`.

In [11]:
try:
    print("PyMTY " + {})
except TypeError as err:
    print("Se capturo el error: {}".format(err))
    raise Exception("No puedes concatenar cadenas y diccionarios!")

Se capturo el error: Can't convert 'dict' object to str implicitly


Exception: No puedes concatenar cadenas y diccionarios!

En Python 3 existe una forma alternativa de programar las excepciones que permite especificar de forma explicita de donde proviene la excepción original.

In [17]:
try:
    print("PyMTY " + {})
except TypeError as err:
    print("Se capturo el error: {}".format(err))
    raise Exception("No puedes concatenar cadenas y diccionarios!") from err

Se capturo el error: Can't convert 'dict' object to str implicitly


Exception: No puedes concatenar cadenas y diccionarios!

### try/except...

El bloque `try/except` puede tener cualquier cantidad de excepts, que se evalúan de arriba hacia abajo, dejando las excepciones más precisas cercas del bloque `try` y las mas genéricas al final, por ejemplo:

In [12]:
class XError(TypeError):
    """
    Error especifico de X
    """
    
try:
   raise XError()
except TypeError:
    print("Se capturo TypeError")
except XError:
    print("Se capturo XError")

Se capturo TypeError


Imprime *"Se capturo TypeError"*, dado que se evalúa primero y la clase `XError` hereda de la clase `TypeError`, en cambio de esta forma:

In [14]:
class XError(TypeError):
    """
    Error especifico de X
    """
try:
   raise XError()
except XError:
    print("Se capturo XError")
except TypeError:
    print("Se capturo TypeError")

Se capturo XError


In [2]:
class XError(TypeError):
    """
    Error especifico de X
    """
try:
   raise XError()
except NotImplementedError:
    print("Not Implemented")
except TypeError:
    print("Se capturo TypeError")

Se capturo TypeError


Imprime *"Se capturo XError"*

### try/except.../else

La siguiente forma casi completa del bloque `try/except` agrega un bloque de código que se ejecuta solamente si no sucedió ninguna excepción en el bloque `try`, éste se delimita con la palabra reservada `else`, solamente se puede tener un solo bloque else para cada bloque de excepciones, donde se requiere de por lo menos esperar una excepción con `except`.

In [3]:
try:
   print("pymty")
except OSError as e:
   print("Error en el sistema operativo!")
else:
   print("Todo normal...")

pymty
Todo normal...


In [4]:
try:
    print("pymty")
    raise OSError("test")
except OSError as e:
    print("Error en el sistema operativo!")
else:
    print("Todo normal...")

pymty
Error en el sistema operativo!


### try/except.../else/finally

Ahora si vamos a conocer el bloque en toda plenitud con el ultimo fragmento que tiene la palabra reservada de `finally` como denominador se ejecuta siempre sin importar si ocurrió o no una excepción en el bloque de `try`, esto es aún y tomando en cuenta que dentro del bloque try se use `return` para regresar un valor dentro de una función, primero ejecuta `finally`, luego regresa de la función en caso de que se use `return`.

In [6]:
try:
    1 / 0
except ZeroDivisionError:
    print("Con que dividiendo por cero...")
except Exception:
    print("Error generico")
else:
    print("Todo muy bien")
finally:
    print("Limpiando memoria, cerrando locks y cosas asi.")

Con que dividiendo por cero...
Limpiando memoria, cerrando locks y cosas asi.


In [5]:
def return_what():
    try:
        print("In try body")
        return 1
    finally:
        print("In finally")
        return 2 # preferencia sobre el return dentro de try

In [6]:
return_what()

In try body
In finally


2

In [9]:
def return_what():
    try:
        print("In try body")
        return 1
    finally:
        print("In finally")
        # sin return en finally, usa el de try.

In [8]:
return_what()

In try body
In finally


1

## 2. Comprehensions

### List comprehensions

In [21]:
del_uno_al_diez = [i for i in range(1, 11)] # <- magic happening
print(del_uno_al_diez)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [8]:
import random
diez_random = [random.random()
                for _ in range(10)] # <- magic happening
print(diez_random)

[0.9179805589915441, 0.9413978643884156, 0.9649443359094384, 0.43684773538149757, 0.9392821519901322, 0.2793101523740499, 0.7577689714320468, 0.8891730895977987, 0.6311330592297092, 0.9283746315246989]


In [ ]:
[<ACCION> for <VARIABLE_OPCIONAL> in <ITERADOR>]

### Set comprehensions

In [22]:
del_uno_al_diez_en_set = {i for i in range(1, 11)} # <- magic happening again!
print(del_uno_al_diez_en_set)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


### Dict comprehensions

In [24]:
_names = [
    'uno', 'dos', 'tres', 'cuatro', 'cinco',
    'seis', 'siete', 'ocho', 'nueve', 'diez'
]

del_uno_al_diez_en_dict = {
    name: i for name, i in zip(_names, range(1, 11)) # <- magic happening again and again!
} 

print(del_uno_al_diez_en_dict)
print(del_uno_al_diez_en_dict['diez'])

{'tres': 3, 'nueve': 9, 'seis': 6, 'ocho': 8, 'cinco': 5, 'siete': 7, 'uno': 1, 'dos': 2, 'diez': 10, 'cuatro': 4}
10


In [14]:
 list(zip([1, 2, 3], [4, 5, 6])) # ((1, 4), (2, 5), (3, 6)))

[(1, 4), (2, 5), (3, 6)]

 ### Generator expression

In [21]:
del_uno_al_diez_gen = (i for i in range(1, 11)) 
# ^ magic happening again and again... and again!
print(del_uno_al_diez_gen)

<generator object <genexpr> at 0x7fe0c807d048>


wut wut?!

In [22]:
next(del_uno_al_diez_gen)

1

In [23]:
next(del_uno_al_diez_gen)

2

In [24]:
for num, i in enumerate(del_uno_al_diez_gen, 10):
    print("Num ", num)
    print(i)

Num  10
3
Num  11
4
Num  12
5
Num  13
6
Num  14
7
Num  15
8
Num  16
9
Num  17
10


In [25]:
next(del_uno_al_diez_gen) # ???

StopIteration: 

### Aplica para todas

**Filtro**

In [32]:
[i for i in range(1, 21) if i % 2]  # nones

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [33]:
[i for i in range(1, 21) if not i % 2]  # pares

[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

**Anidado de expresiones**

In [36]:
[i * x for x in range(1, 3)
       for i in range(1, 21) if i % 2]

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 2, 6, 10, 14, 18, 22, 26, 30, 34, 38]

In [43]:
[ '{} => {}'.format(z, i * x) # [1 * 5, 3 * 5, 5 * 5, ...]
   for z in ('ciclo 1', 'ciclo 2', 'ciclo 3')
    for x in range(1, 10) if x == 5   # [5]
       for i in range(1, 10) if i % 2] # [1, 3, 5, 7, 9]

['ciclo 1 => 5',
 'ciclo 1 => 15',
 'ciclo 1 => 25',
 'ciclo 1 => 35',
 'ciclo 1 => 45',
 'ciclo 2 => 5',
 'ciclo 2 => 15',
 'ciclo 2 => 25',
 'ciclo 2 => 35',
 'ciclo 2 => 45',
 'ciclo 3 => 5',
 'ciclo 3 => 15',
 'ciclo 3 => 25',
 'ciclo 3 => 35',
 'ciclo 3 => 45']

Lo anterior de una forma tradicional, sería:

In [47]:
resultado = []
for z in ('ciclo 1', 'ciclo 2', 'ciclo 3'):
    for x in range(1, 10):
        if x == 5:
            for i in range(1, 10):
                if i % 2:
                    resultado.append('{} => {}'.format(z, i * x))
resultado

['ciclo 1 => 5',
 'ciclo 1 => 15',
 'ciclo 1 => 25',
 'ciclo 1 => 35',
 'ciclo 1 => 45',
 'ciclo 2 => 5',
 'ciclo 2 => 15',
 'ciclo 2 => 25',
 'ciclo 2 => 35',
 'ciclo 2 => 45',
 'ciclo 3 => 5',
 'ciclo 3 => 15',
 'ciclo 3 => 25',
 'ciclo 3 => 35',
 'ciclo 3 => 45']

**Extra, usando el *operador terniario* **

In [48]:
[i if i.endswith('x') else i + 'y'
 for i in ('ax', 'b', 'cx', 'd')]

['ax', 'by', 'cx', 'dy']

Dicho todo lo anterior...

   **<font color="red">¡¡NO ABUSEN DE LOS COMPREHENSIONS!!</font>**

## 3. Generadores y corrutinas

In [37]:
import random

## generador 
def own_range(from_, to):
    """
    Generator that half-reimplements the lazy range in Python 3.
    """
    if to < from_:
        raise Exception("`to` cannot be smaller than `from_`")
    pos = from_
    while pos <= to:
        yield pos # congelada
        pos += 1 # desconge
        
        
def conversation(name):
    """
    Chatty corrutine
    """
    phrases = ["Interesante.", "Ya veo.", "¿Y que más?"]
    response = yield "Hola {}!".format(name)
    while True:
        if response in ('adios', 'bye'):
            break
        else:
            response = yield phrases[random.randint(0, 2)]


In [38]:
own_range

<function __main__.own_range>

In [28]:
range_ = own_range(1, 10)
range_

<generator object own_range at 0x7fe0c80aee60>

In [29]:
next(range_)

1

In [30]:
next(range_)

2

In [31]:
for i in range_:
    print(i)

3
4
5
6
7
8
9
10


In [32]:
this_is_wrong = own_range(10, 1)
this_is_wrong

<generator object own_range at 0x7fe0c80aea98>

In [33]:
next(this_is_wrong)

Exception: `to` cannot be smaller than `from_`

In [39]:
conv = conversation('Joel')
conv

<generator object conversation at 0x7fe0c807dc50>

In [40]:
conv.send(None) # se inicializa con None o se llama a next

'Hola Joel!'

In [41]:
conv.send("Ayer llovio!")

'Ya veo.'

In [61]:
conv.send('Que interesante no?')

'Interesante.'

In [62]:
conv.send("Pues no mucho...")

'¿Y que más?'

In [63]:
conv.send('WTF?')

'¿Y que más?'

In [64]:
conv.send("bye") # tambien funcionaria conv.close()

StopIteration: 

Python 3 adicionalmente soporta `yield from` para iterar sub-generadores. 

Para más información sobre corrutinas: http://www.dabeaz.com/coroutines/Coroutines.pdf

Mejor aún el nuevo módulo `asyncio`: https://docs.python.org/3/library/asyncio.html


## 4. Lambdas

Las *lambdas* es otro de los recursos usados en la programación funcional.

Usualmente se les conoce como **funciones anónimas**.

La implementación en Python, esta diseñada para no fomentar el uso extensivo de estás, principalmente por que es complicado de *debuggear* el código que abusa de las lambdas.

Una lambda tiene todas las caracteristicas de las funciones que ya vimos (incluyendo el asunto del scope y argumentos), pero tiene una *pequeña* limitante, **solo soporta una expresion en el cuerpo de la función**.

Se definen por medio de la palabra reservada **lambda**.

In [66]:
lambda a, b: a + b

<function __main__.<lambda>>

In [67]:
(lambda a, b: a + b)(10, 20)

30

In [68]:
variable_comun = lambda a, b: a + b

In [69]:
variable_comun(10, 20)

30

In [71]:
variable_comun

<function __main__.<lambda>>

**¡¡No tiene nombre!! ^^^^** 

In [73]:
# no hagan esto en código que les importe
factorizador = lambda func, factor=10: \
                  lambda *args, **kwargs: \
                       func(*[a * factor for a in args], **kwargs) \
                               if all((isinstance(a, int) for a in args)) else \
                                func(*args, **kwargs)

In [74]:
factorizador

<function __main__.<lambda>>

In [75]:
@factorizador
def printer(*args, **kwargs):
    print("Mis args son: ", args)
    print("Mis kwargs son: ", kwargs)
printer()

Mis args son:  ()
Mis kwargs son:  {}


In [76]:
printer(1, 2, 3, 4, 5) # todos son enteros

Mis args son:  (10, 20, 30, 40, 50)
Mis kwargs son:  {}


In [77]:
printer(1, 2, 3, 4, 'Joel') # una cadena!!

Mis args son:  (1, 2, 3, 4, 'Joel')
Mis kwargs son:  {}


## 5. Módulos

Los módulos son una abstracción del lenguaje que sirve para organizar el código, permiten distribuir el código en distintos archivos y es un bloque fundamental para distribuir el código para terceras partes.

Principalmente son **necesarios** por que es muy dificil recordar todas esas lineas de código en la cabeza.

La representación tradicional de un módulo de python en el sistema de archivo es en los archivos que tienen extension `.py`. Es decir, todo script en python en un módulo en si.

Cabe destacar, que esa representación no es la única. Tambien pueden esta en forma de directorios (paquetes), en forma de zip o en *shared objects* para las extensiones nativas (como numpy).

Python es un lenguaje *con baterias incluidas*, es decir, incluye mucha funcionalidad para una gran cantida de tareas, para más información revisen la librerías estándar de python: https://docs.python.org/3/library/

### Importando con `import`

In [127]:
import collections

import webbrowser as chrome

from xml import etree

from math import ceil, floor, pi

#from math.universe_x.a.b.c.d import suma
#from math.universe_x  as suma_d

from array import array as list_a

from enum import * 

La sentencía `import` no tiene una restricción sobre donde puede ejecutarse, más se considera buen gusto el realizar los **imports al inicio del archivo.**

In [78]:
def get_pi():
    from math import pi
    return pi

In [79]:
get_pi()

3.141592653589793

## 6. Paquetes

El termino paquetes se refiere a la distribución de código de Python en una estructura especifica en directorios , relativos al directorio de trabajo, la variable de entorno `PYTHONPATH` y finalmente a la lista donde se termina representando todos estos lugares dentro del módulo `sys`.

In [81]:
import sys

sys.path

['',
 '/home/joe/anaconda3/envs/databootcamp/lib/python35.zip',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/plat-linux',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/lib-dynload',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/site-packages/Sphinx-1.3.5-py3.5.egg',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/site-packages/setuptools-20.3-py3.5.egg',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/site-packages',
 '/home/joe/anaconda3/envs/databootcamp/lib/python3.5/site-packages/IPython/extensions',
 '/home/joe/.ipython']

In [85]:
import math
import sys
sys.modules['math'] is math

True

In [86]:
!ls -l

total 184
-rw-rw-r-- 1 joe joe  21731 jun 11 03:58 1. Sesión-1.ipynb
-rw-rw-r-- 1 joe joe   8186 jun 11 03:58 1. Sesión-2.ipynb
-rw-rw-r-- 1 joe joe 133510 jun 11 13:01 1. Sesión-3.ipynb
drwxrwxr-x 3 joe joe     62 jun  8 17:35 mipaquete
drwxrwxr-x 5 joe joe     45 jun 11 03:50 nssample
-rw-rw-r-- 1 joe joe  18409 jun 11 03:59 Workspace-sesion-1.ipynb


In [87]:
!tree mipaquete

mipaquete
├── __init__.py
├── __pycache__
│   ├── __init__.cpython-35.pyc
│   └── saludos.cpython-35.pyc
└── saludos.py

1 directory, 4 files


In [102]:
#import mipaquete
from mipaquete import saludos
saludos.hola()

Hola alumnos


In [137]:
!tree nssample

nssample
├── part1
│   └── frutas
│       └── manzanas.py
├── part2
│   └── frutas
│       └── naranjas.py
└── part3
    └── frutas
        └── uvas.py

6 directories, 3 files


In [103]:
sys.path.append('nssample/part1')

In [105]:
import frutas

In [106]:
frutas

<module 'frutas' (namespace)>

In [107]:
import frutas.manzanas
frutas.manzanas

<module 'frutas.manzanas' from 'nssample/part1/frutas/manzanas.py'>

In [108]:
import frutas.naranjas # esto va a fallar...

ImportError: No module named 'frutas.naranjas'

In [109]:
sys.path.append('nssample/part2')
import frutas.naranjas # pero ya no
frutas.naranjas

<module 'frutas.naranjas' from 'nssample/part2/frutas/naranjas.py'>

In [110]:
print(frutas.naranjas.get())
print(frutas.manzanas.get())

Naranjas!
Manzanas


In [111]:
# lo mismo para las uvas

sys.path.append('nssample/part3')
from frutas import uvas

In [112]:
uvas

<module 'frutas.uvas' from 'nssample/part3/frutas/uvas.py'>

In [113]:
uvas.get()

'Uvas!'

<font color="red">¡NOTA IMPORTANTE!</font>

evitar el uso de:

    from module import *

## 7. Distribuyendo el código

Distutils (parte de la librería estándar): https://docs.python.org/3/library/distutils.html

Setuptools (módulo que es un super-set de distuils): https://setuptools.readthedocs.io/en/latest/

Herramientas más utilizada para instalar paquetes en python: https://pypi.python.org/pypi/pip

Una de las herramientas más populares (por el momento) para crear el "boilerplate" de proyectos comunes: https://pypi.python.org/pypi/cookiecutter/1.4.0

### Ejemplo de  `setup.py` 

```python
# chardet's setup.py
from distutils.core import setup

setup(
    name = "chardet",
    packages = ["chardet"],
    version = "1.0.2",
    description = "Universal encoding detector",
    author = "Mark Pilgrim",
    author_email = "mark@diveintomark.org",
    url = "http://chardet.feedparser.org/",
    download_url = "http://chardet.feedparser.org/download/python3-chardet-1.0.1.tgz",
    keywords = ["encoding", "i18n", "xml"],
    classifiers = [
        "Programming Language :: Python",
        "Programming Language :: Python :: 3",
        "Development Status :: 4 - Beta",
        "Environment :: Other Environment",
        "Intended Audience :: Developers",
        "License :: OSI Approved :: GNU Library or Lesser General Public License (LGPL)",
        "Operating System :: OS Independent",
        "Topic :: Software Development :: Libraries :: Python Modules",
        "Topic :: Text Processing :: Linguistic",
        ],
    long_description = """\
Universal character encoding detector
-------------------------------------

Detects
 - ASCII, UTF-8, UTF-16 (2 variants), UTF-32 (4 variants)
 - Big5, GB2312, EUC-TW, HZ-GB-2312, ISO-2022-CN (Traditional and Simplified Chinese)
 - EUC-JP, SHIFT_JIS, ISO-2022-JP (Japanese)
 - EUC-KR, ISO-2022-KR (Korean)
 - KOI8-R, MacCyrillic, IBM855, IBM866, ISO-8859-5, windows-1251 (Cyrillic)
 - ISO-8859-2, windows-1250 (Hungarian)
 - ISO-8859-5, windows-1251 (Bulgarian)
 - windows-1252 (English)
 - ISO-8859-7, windows-1253 (Greek)
 - ISO-8859-8, windows-1255 (Visual and Logical Hebrew)
 - TIS-620 (Thai)

This version requires Python 3 or later; a Python 2 version is available separately.
"""
)

```